# Algorithms


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/UoB-HPC/minicombust/blob/main/docs/Algorithms.ipynb)

```{toctree}
```



In [76]:
import numpy as np
from numpy.typing import NDArray
from typing import Final

from minicombust import Cell, Face, Geometry, T
from minicombust.utils import minmax
from minicombust.algorithms  import limit_slope

# The main flow solver loop

## Time integration
MiniCombust uses the Implicit Euler Method for time integration (see [Peric](./Resources.ipynb#peric), 6.3.2)

## The SIMPLE algorithm
The approach taken in MiniCombust is the same as in Dolyn: Spalding's [SIMPLE (Semi-implicit Method for Pressure-Linked Equations)](https://en.wikipedia.org/wiki/SIMPLE_algorithm) algorithm with corrector steps. See Peric , Section 8.8, and also Chapter 7, for a detailed overview of SIMPLE.

The SIMPLE algorithm solves the momentum equations for velocity using a _guess_ for pressure field and face mass fluxes, and then updates mass fluxes with the new velocity values. But these will not automatically satisfy the continuity equations,
so we subsequently calculate a flux correction (solving the pressure equation), and apply the correction to the new velocity values.

Simple uses a _sequential_ approach for solving the system of discretised PDEs (and particle tracking), performing calculations for the governing equations one at a time. Since the PDEs are non-linear, in each
step we will use Picard iteration to solve the non-linearities

The Dolfyn approach uses a co-located grid (pressure and velocity fields' points at the same coordinates). However, this co-located approach (as approach to the other commonly encountered approach: staggered grids), results in
a numerical chequerboarding effect, and corrector steps must be used to overcome this. Dolfyn uses Rhie and Chow's pressure dissipation term, as described in Section 8.8 of [Peric](./Resources.ipynb#peric).

## The main flow solver program loop is shown below


In [2]:
def is_converged(residuals) -> bool:
  """
    If the max of any residual is < ResMax then we have converged and can stop the 'outer loop' (success)
    Note that we don't test the epsilon residual, because it has no meaning
  """
  pass


def is_diverged(residuals) -> bool:
  """
  If the max of any reisual is > 1e18, then we have diverged and can stop (fail)
  Note that we don't test the epsilon residual, because it has no meaning
  """
  pass


def calulate_uvw() -> bool:
  # Calulate new u,v,w values using guess for the mass flux
  # Calculate flux through all inner faces, and process boundary conditioons
  # Set source terms (Su, Sv, Sw) to initial value based on body force (enthalpy) calcs and 
  #    previous value of pressure gradient
  # Set coefficient matrix entries (value of Au, Av, Aw) based on modifying previous U,V,W values and density
  #     and source terms (Su, Sv, Sw) 
  #     f = cell.density() * cell.volume() * 1/dt; Su += f * U_prev etc.
  # Set up 3 linear systems Ax = S using (Au, Av, Aw) and (U,V,W) and source terms (Su, Sv, Sw) and underrelaxation factors
  # Call PetSc solver for systems to solve for new U,V,W

def calculate_pressure() -> bool:
  # calculated pressure correction
  # new mass flux values using cell densituis, U,V,W values and gradients, with interpolation between faces' adjacent cells
  # See Peric eq. 8.56
  # 

def one_outer_step() -> bool:
  # precompute gradients of velocity component fields (U, V, W) and pressure fields (P), because we use 
  # them in several places
  # result: whether to keep going or not based on convergence or divergence

  dUdX = gradient(U)
  dVdX = gradient(V)
  dWdX = gradient(W)
  dPdX = gradient(P)

  calculate_UVW()
  calculate_pressure(dppmax)

  update_boundary_conditions()
  
  solve_turbulent_energy()
  solve_turbulent_dissipation()
  solve_viscosity()
  solve_enthalpy()
  
  solve_all_other_scalars() # The combustion-specific stuff here!

  # If we hadn't split particle and flow solver into different ranks, then
  #particle_solver()
  #evaluate_particle_sensors()

  return is_converged(residuals) or is_diverged(residuals)

def do_outer_loop():
  keep_going = True
  iteration = 0
  while keep_going and iteration < MAX_ITERATIONS:
    keep_going = one_outer_step()

 

In [1]:
time = 0
for timestep in range(num_timesteps):
  time += dt

  # Copy the old values for variables and fields into a (t-1) field for 
  # solving transient terms
  copy_values_to_old_values()

  do_outer_loop()

NameError: name 'num_timesteps' is not defined

### The "Guess"
The guess for each kind of variable is taken from:

| Variable | Guess | Description |
|----|-----|---|
| $u$ | 0 | Component of velocity |
| $v$ | 0 | Component of velocity |
| $w$ | 0 | Component of velocity |
| $p$ | 0 | Pressure |
| $e$ | 0 | Energy? |
| $T$ | 293.0 | Temperature |
| $Sc$ | 0 | Any other scalar |
| $Den$ | 0 | Density?? |
| $PP$ | 0| ??? |


# Linear Solvers

Since MiniCombust relies on very large (very distributed) problems, the linear solvers used to solve the systems $A\mathbf{x} = \mathbf{b}$ (with A very sparse) must scale. As a results, we use iterative solvers.
MiniCombust doesn't implement any linear solvers, but uses the PetSc library. Specifically, we use the HYPRE functionality in PETSc to perform Multigrid ?

In addition, the discretise equations have non-linearities, so we must use several sweeps (updating the source terms and coefficients using current iteration values of $x$)
Unlike with structured grid/stencil problems, $A$ does not have a specific band struture.

## What is each system Ax = b? Where does it come from?
The same pattern happens for every scalar (or component of velocity that we calculate)
We resolve turbulence, update the flux values, find the specific under-relaxation factor for this variable, 
Then call SetupMatrixA then SolveMatrixA, 
Check feasibility of solution (checking whether enthalpy and the scalar value are within ddefined ranges, and clipping to max and min tolerancecs),
update boundaries

SetupMatrixA buiilds up the entries of a sparse matrix A from given set of coefficient values, modifying them
and the soure terms for underrelaxation with

$$
U^{NEW,USED}_{i} = U^{OLD}_{i} + \alpha \big( U^{NEW,PREDICTED}_{i} - U^{OLD}_{i} \big)
$$

In each case $A\textbf{x} = \textbf{b}$:

* $A$ is the modified matrix of coeffients. It is sparse, and only has entries far each face, i.e. it is $(n_cell+n_boundary) x (n_cel+n_boundary)$, but stored as a CSR matrix
* $b$ is the dense vector of all the source terms ($n_cell + n_boundary$ elements)
* $x$ is the scalar variable to solve for (1 element per cell)


So, for example, as part of `SolveUVW`, we set up a linear system $A_U\textbf{u} =\textbf{s_u}$ where
* $A_u$ is the coefficients calculated for the U velocity components
* We will be solving for the new values of $u$
* $s_u$ is built by adding all the source terms in the velocity equations, taking into account previous timesteps as per [Peric](./Resources.ipynb#peric), 6.3.2 Implicit Euler Method

In [2]:
subroutine SetUpMatrixA(ivar,URFactor,A,Phi,S)

   use constants
   use geometry
   use variables
   use scalars 

   real, dimension(Ncel)      :: A, S 
   real, dimension(Ncel+Nbnd) :: Phi 

   if( Debug > 3 ) write(*,*)'*** SetUpMatrixA  ',Variable(ivar),URFactor

   RURF = 1./URFactor
  
   Res(1:Ncel) = 0.0
   
   ia = 0    
   do i=1,Ncel

     app = A(i)

     do j=1,NFaces(i)
       k  = CFace(i,j)
       ip = Face(k)%cell1
       in = Face(k)%cell2
       if( in > 0 )then
           ! internal
           if( ip == i )then
             aface   = RFace(k,2)
             ia = ia + 1
             Acoo(ia) = DBLE( aface )
             Arow(ia) = i
             Acol(ia) = in
             
             Res(i) = Res(i) - aface*Phi(in)
           elseif( in == i )then
             aface = RFace(k,1)
             ia = ia + 1
             Acoo(ia) = DBLE( aface )
             Arow(ia) = i
             Acol(ia) = ip

             Res(i) = Res(i) - aface*Phi(ip)
           else
             write(*,*)'+ internal error: assembly A-matrix. in=',in
             write(*,*)'+ in SetUpMatrixA for ',Variable(ivar)
           endif
           app = app - aface
       endif
     end do
     A(i)  = app * RURF  
     S(i)  = S(i) + (1.0-URFactor)*A(i)*Phi(i)

     ia = ia + 1
     Acoo(ia) = DBLE( A(i) )
     Arow(ia) = i
     Acol(ia) = i

     Res(i) = Res(i) + S(i) - A(i)*Phi(i)
   end do

   if( ia /= NNZ ) write(*,*)'+ error: SetUpMatrixA: NNZ =',ia,' =/=',NNZ

   !
   ! norm 0: maxval res(i)
   ! norm 1: sum(abs(res(:)))
   ! norm 2: sqrt(sum(res(:)**2))
   !
  !Res0 = sum(abs(Res)) * ResiNorm(iVar)
   Res0 = sqrt(sum(abs(Res(1:Ncel)**2))) * ResiNorm(iVar) 
   
!   if( Res0 > 1.e8 ) Res0 = 10.0
    
   write(IOdbg,*)'Res0:',Res0,ResiNorm(iVar),ivar,Variable(ivar)
   
   if( Debug > 3 ) write(*,*)'=== SetUpMatrixA' 

end subroutine SetUpMatrixA

# Testing whether a particle is in a cell

In [ ]:
# Determining whether a particle is in a cell
# From Dolfyn subroutie "ParticleInCell" (particles.f90)
"""
We use the cell face normals (positive out) and a normalised position vector from the particle's position to the face centre.
Calculate dot product with cell face centre.
Result: 0 on face, positive inside, negative outside.

Check for all cell faces whether any one is negative.
"""

def normalise(v: T3) -> T3:
    norm=np.linalg.norm(v, ord=1)
    if norm==0:
        norm=np.finfo(v.dtype).eps
    return v/norm


def is_particle_in_cell(particle_coords: T3, local_cell_id) -> bool:
  cell_num_faces = cells.num_faces(local_cell_id)
  for face_id in cells.faces(local_cell_id):
      cell1_id, cell2_id 

logical function ParticleInCell(Xp,ic,ifound,Sloppy)
!========================================================================

   use geometry, only: Nfaces, CFace, Face  

   integer, intent(IN)  :: ic
   real, intent(IN)     :: Xp(3), Sloppy
   integer, intent(OUT) :: ifound

   integer              :: j, k, ip, in
   real                 :: Xf(3), Xn(3)
   logical              :: flag 

   flag   = .true.                          ! assume it is in cell ic
   icnt   = 0
   ifound = -1

   do j=1,NFaces(ic)
     k  = CFace(ic,j)
     ip = Face(k)%cell1
     in = Face(k)%cell2
     if( ip == ic )then 
       Xn =  Face(k)%n 
     else if( in == ic )then
       Xn = -Face(k)%n 
     else
       write(*,*)'Error in particle in cell'
     endif

     Xf = Face(k)%x - Xp                  ! vector from particle to face centre

     call normalise(Xf)
     call normalise(Xn)                   ! normal is not normalised normalize

     dotp  = dot_product( Xf , Xn )       ! dot product

!   if( ic == 338 )write(*,*)'x>',j,k,dotp,sloppy
!   if( ic == 218 )write(*,*)'x2>',j,k,dotp,sloppy
     if( dotp < Sloppy )then
       flag   = .false.                   ! Oops! outside of face! (0.0 is on the face)
       icnt   = icnt + 1
       ifound = k

       !if( ic == 225 )hoek = acos( dotp )*180./3.1415927  
       !if( ic == 225 )write(*,*)'f:',ic,j,dotp,' hoek:', hoek,flag,sloppy  
!   write(*,*)'f:',ic,'=>',j,flag,dotp,acos( dotp )*180./3.1415927,sloppy
     endif

   end do
!   if( flag == .false. )write(*,*)'f:',ic,'=>',flag,k

   !if( icnt == 1 )then
   !  write(*,*)'one face failed'
   !else if( icnt > 1 )then
   !  write(*,*)'multiple faces failed ',icnt
   !  ifound = -1
   !endif

   ParticleInCell = flag

end function ParticleInCell


# Calculating gradients
MiniCombust uses [Gauss's Divergence Theorem](https://en.wikipedia.org/wiki/Divergence_theorem) to calculate the gradients in diffusive flux terms.
The divergence theorem allows us to express the flux of a vector field through the surface in terms of the divergence of the field in the volume enclosed.

$$
\int_V (\nabla \cdot \mathbf{\Phi}) dV = \oint_S \mathbf{\Phi} d\vec{s}
$$

or in terms of discrete faces:

$$
(grad \mathbf{\Phi})_P \approx \frac{1}{V_P}\sum_{j=1}^{n}\Phi_j \vec{s_j}
$$

with $\Phi_j$ the value stored at the centre of face $j$

During gradient calculation, we also perform a deferred correction by adjusting the coordinates by weighting the adjacent cells' contributions using the face's interpolation factor property ($\lambda$). This is described in [Peric](Resources.ipynb/#peric) in §8.6.2 (Approximation of Diffusive fluxes). Note that we limit the number of passes of gradient estimation to 2.

Once the gradient has been calculated, we can apply an appropriate slope limiter, in case of bad geometries,
and MiniCombust only supports the approach in [Venkatakrishnan1993].

## Slope limiter: Venkatakrishnan (1993)
The only slope limiter supported by MiniCombust is the approach in  AIAA-93-0880, _On the accuracy of limiters and convergence to steady state solutions_, V.Venkatakrishnan, 1993.

We test the gradient against the original 'neighbour values' as with the approach defined in Barth and Jespersen [ref], and then 
limit it with:

$$
\phi(y) = \frac{y^2 + 2y}{y^2 + y + 2}
$$

The neighbour values we use are those at the surrounding _nodes_ (rather than cell centres or face centres). This approach
is a compromise between overshoot when using face centre values and undershoot when using cell centre values.



In [80]:
from IPython.display import Code
Code(filename="minicombust/algorithms.py", language="python")

import numba
import numpy as np
from typing import Final
from numpy.typing import NDArray
from .data_structures import Geometry, T
from .utils import minmax

# Adapted from Dolfyn gradients.f90, GradientPhiLimiterVenkatarishnan
# MiniCombust only supports "approach 3" (using nodes instead of face centres for limiter)
def limit_slope(Φ: NDArray[T], dΦdX: NDArray[T], mesh: Final[Geometry]) -> NDArray[T]:
    assert(len(Φ) == mesh.num_cells + mesh.num_boundaries) 
    assert(Φ.ndim == 1) # It's a 1D tensor

    assert(Φ.ndim == 1) # It's a 2D tensor
    assert(dΦdX.shape[0] == Φ.shape[0]) # that's the same length as Φ
    assert(dΦdX.shape[1] == 3) # Its nx3

    cells = Geometry.cells

    for local_cell_id, cell in enumerate(mesh.cells):
      Φ_for_this_cell = Φ[local_cell_id]
      neighbour_Φs = Φ[cells.neighbours_with_offsetted_boundary_ids(local_cell_id)]
      Φ_min, Φ_max = minmax(neighbour_Φs)

      # Deterine the smallest and largest deltas to neighbours for this cell
      Δ_min = Φ_min - Φ_for_this_cell
      Δ_max = Φ_max - Φ_for_this_cell        

      # Determine smallest limiter α considering slope to node coords
      surrounding_vertices_coords = cells.vertex_coords(local_cell_id)
      cell_centre = cells.coords[local_cell_id]
      ds = surrounding_vertices_coords - cell_centre
      Δ_faces = np.dot(dΦdX[local_cell_id, :], ds)

      @numba.jit
      def venkatarishnan_factor(Δ_faces: NDArray[T], Δ_min: T, Δ_max: T):
        α = np.arraylike(Δ_faces)
        min_α = np.iinfo(T).max # i.e. MAX_FLOAT
        for Δ_face, i in enumerate(Δ_faces):
          if np.abs(Δ_face) < 1.e-6:
            r = 1000.0
          elif Δ_face > 0.0:
            r = Δ_max / Δ_face
          else:
            r = Δ_min / Δ_face
          α = (r**2+2.0*r)/(r**2+r+2.0)
          min_α = α if α < min_α else min_α
        return min_α
      
      α = venkatarishnan_factor(Δ_faces, Δ_min, Δ_max)
      dΦdX[local_cell_id,:] *= α
    return dΦdX